## Analysis of blood glucose data to assess the reproducibility of the post-prandial blood glucose response (PPGR)
### Process the raw blood glucose data to generate 'glucose events' (initial nadir / peak / secondary nadir)

1. Preprocess the raw data
2. Slice the blood glucose data by the meal_ts or the bolus_start_ts to ascertain the gluocse event linked to food intake
3. Analysis te blood gluocse events for the relevant inforamtion to create a 'fingerprint' for each event
    * Split into meal times (breakfast, lunch, dinner, overnight)
    * Isolate the initial low glucose value, the peak glucose value and the secondary low glucose value of each glucose event
    * Calculate the iAUC (positive, negative and total) for each gluocse event
    * Determine the gradient of the glucose rise and drop (linear regression)
    * Calculate the lapse time from initial low to peak, peak to secondary low and the total lapse time of the glucose event
    * Ascertain the  'noise' in each of the glucose events (Delta)
4. Determine the similarity of the glucose events using a Pearson's correlation
5. Group glucose events by similarity
6. Pull into database with the following headings:
    * Day_Name	
    * Day_number	
    * exp_week	
    * time_period_name	
    * time_period_num	
    * cluster_number	
    * Initial_Low	
    * iAUC_pos	
    * iAUC_neg	
    * AUC_Tot	
    * ROA	
    * ROD	
    * glucose_rise	
    * glucose_drop	
    * peak_value	
    * Secondary_Low	
    * Lapse_to_peak	
    * Peak_to_sec_low	
    * Total_time_lapse	
    * Variance_pos	
    * Variance_neg	
    * Carbs

### Import/load necessary libraries 

In [367]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import datetime
from datetime import timedelta
from statistics import mode
from scipy.signal import find_peaks

%matplotlib inline

### Load and preprocess necessary data sets

In [368]:
def read_data(filename):
    unfiltered = pd.read_csv(os.path.join("CSV Files", filename))
    unfiltered['glucose_level_ts'] = pd.to_datetime(unfiltered['glucose_level_ts'], dayfirst=True)
    unfiltered['bolus_start_ts'] = pd.to_datetime(unfiltered['bolus_start_ts'], dayfirst=True)
    unfiltered['meal_ts'] = pd.to_datetime(unfiltered['meal_ts'], dayfirst=True)
    unfiltered.drop('glucose_level_mmol/L', axis=1, inplace=True)
    return unfiltered
     

In [369]:
def find_closest_timestamp(bolus_start_ts, glucose_level_ts):
    # Convert bolus_start_ts to datetime, handling errors with 'coerce'
    bolus_start_ts = pd.to_datetime(bolus_start_ts, dayfirst=True, errors='coerce')

    # Handle NaN and float values
    if pd.isnull(bolus_start_ts) or isinstance(bolus_start_ts, float):
        return None

    # Convert bolus_start_ts to a pandas Timestamp object
    bolus_start_ts = pd.Timestamp(bolus_start_ts)

    # Convert bolus_start_ts and glucose_level_ts to NumPy datetime arrays
    bolus_start_ts = np.array(bolus_start_ts, dtype='datetime64')
    glucose_level_ts = np.array(glucose_level_ts, dtype='datetime64')

    closest_index = np.argmin(np.abs(glucose_level_ts - bolus_start_ts))

    return glucose_level_ts[closest_index]

In [370]:
def slice_data(data, time_window, n_samples, bolus_start_ts):
    glucose_timestamps = np.array(data['glucose_level_ts'], dtype='datetime64')
    glucose_values = np.array(data['glucose_level'], dtype=float)
    bolus_timestamps = np.array(data['bolus_start_ts'], dtype='datetime64')
    meal_timestamps = np.array(data['meal_ts'], dtype='datetime64')

    closest_bolus_glucose_ts = find_closest_timestamp(bolus_start_ts, glucose_timestamps)
    closest_meal_glucose_ts = find_closest_timestamp(data['meal_ts'].iloc[0], glucose_timestamps)

    # Determine which timestamp to use based on availability
    if pd.notnull(closest_bolus_glucose_ts):
        selected_glucose_ts = closest_bolus_glucose_ts
    elif pd.notnull(closest_meal_glucose_ts):
        selected_glucose_ts = closest_meal_glucose_ts
    else:
        return None, None

    # Check if selected_glucose_ts is NaT and return None if it is
    if pd.isnull(selected_glucose_ts) or pd.isnull(bolus_start_ts):
        return None, None

    time_difference = abs(bolus_start_ts - selected_glucose_ts)

    # Check if the selected glucose timestamp is within the time window
    if time_difference <= pd.Timedelta(minutes=time_window):
        start_index = np.where(glucose_timestamps == selected_glucose_ts)[0][0]
        end_index = start_index + n_samples
        glucose_sliced = glucose_values[start_index:end_index]
        glucose_seg_ts = glucose_timestamps[start_index:end_index]
        return glucose_sliced, glucose_seg_ts
    else:
        return None, None

In [371]:
# Define function to check for correction doses
def is_correction_dose(bolus_start_ts, meal_ts, correction_window):
    if pd.notnull(meal_ts):
        # Convert both timestamps to the same format before calculating time difference
        main_dose = np.datetime64(bolus_start_ts)
        meal_ts = np.datetime64(meal_ts)
        time_difference = (main_dose - meal_ts).astype('timedelta64[m]').astype(int)
        return correction_window[0] <= time_difference <= correction_window[1]
    return False  # Return False if meal_ts is missing (NaN)


In [372]:
unfiltered = read_data('544_data.csv')

In [373]:
unfiltered.head()

,glucose_level_ts,glucose_level_mg/dL,carbs_g,meal_ts,meal_type,bolus_dose,bolus_start_ts,bolus_end_ts,bolus_type
0,2020-05-11 00:02:00,129,135.0,2020-05-11 11:25:00,Lunch,7.1,2020-05-11 08:24:00,11/05/2020 08:24,normal
1,2020-05-11 00:07:00,128,100.0,2020-05-11 17:16:00,Dinner,16.8,2020-05-11 11:24:00,11/05/2020 11:24,normal
2,2020-05-11 00:12:00,129,42.0,2020-05-11 21:26:00,Snack,14.2,2020-05-11 17:19:00,11/05/2020 17:19,normal
3,2020-05-11 00:17:00,131,37.0,2020-05-12 08:02:00,Breakfast,7.5,2020-05-11 21:23:00,11/05/2020 21:23,normal
4,2020-05-11 00:22:00,133,78.0,2020-05-12 12:07:00,Lunch,6.9,2020-05-12 08:00:00,12/05/2020 08:00,normal


In [374]:
# Select glucose level and timestamp features
features = ['glucose_level_ts', 'glucose_level_mg/dL', 'bolus_start_ts', 'meal_ts']
data = unfiltered[features].copy()

# Map day of week names to day of week numbers
data['glucose_level'] = data['glucose_level_mg/dL'] / 18
data['glucose_level'] = data['glucose_level'].round(decimals=1)

# Set time step in minutes
time_step = data['glucose_level_ts'].diff().mode()[0] / timedelta(minutes=1)

# Calculate rate of change
data['bg_rate_of_change'] = data['glucose_level'].diff() / time_step

#Drop the 'glucose_level_mg/dL' column if desired
data.drop('glucose_level_mg/dL', axis=1, inplace=True)


In [375]:
# Get the column names
#columns = data.columns.tolist()

# Move the 3rd column to the first position
#columns = [columns[2]] + columns[:2] + columns[3:]

# Reorder the columns in the DataFrame
#data = data[columns]


In [376]:
data.head()

,glucose_level_ts,bolus_start_ts,meal_ts,glucose_level,bg_rate_of_change
0,2020-05-11 00:02:00,2020-05-11 08:24:00,2020-05-11 11:25:00,7.2,NaN
1,2020-05-11 00:07:00,2020-05-11 11:24:00,2020-05-11 17:16:00,7.1,-0.02
2,2020-05-11 00:12:00,2020-05-11 17:19:00,2020-05-11 21:26:00,7.2,0.02
3,2020-05-11 00:17:00,2020-05-11 21:23:00,2020-05-12 08:02:00,7.3,0.02
4,2020-05-11 00:22:00,2020-05-12 08:00:00,2020-05-12 12:07:00,7.4,0.02


In [377]:
#timestamp = data['glucose_level_ts'].to_numpy('datetime64')
#glucose = data['glucose_level'].to_numpy('float32')


In [378]:
#elements_per_day = int(24 * 60 / time_step)  # Number of elements in a day
#segments_per_day = 4  # Number of segments per day

# Calculate the number of elements in each segment
#elements_per_segment = int(elements_per_day / segments_per_day)


### Isolate the peak and nadir (low) points in the set windows

In [379]:
# Set the time window to 5 minutes (300 seconds)
time_window = 5

# Set the number of samples (for slicing) as desired
n_samples = 48

# Define the correction dose window (e.g., 30 minutes)
correction_window = (10, 30)

# List to store glucose segments for each bolus event
glucose_sliced_segments = []

# List to store timestamps for each glucose segment
glucose_seg_ts_segments = []


In [380]:
# Iterate through each bolus event
for bolus_start_ts in data['bolus_start_ts']:
    # Slice the data based on the closest glucose timestamp to the bolus timestamp
    glucose_sliced, glucose_seg_ts = slice_data(data, time_window, n_samples, bolus_start_ts)
    
    # Check if any segments were found and if it's not a correction dose
    if glucose_sliced is not None and len(glucose_sliced) > 0:
        # Filter out non-numeric values from the glucose_sliced array
        glucose_sliced_numeric = glucose_sliced[~np.isnan(glucose_sliced)]
        
        # Convert glucose_sliced_numeric to a NumPy datetime array
        glucose_sliced_datetime = np.array(glucose_sliced_numeric, dtype='datetime64')
        
        # Find the nearest meal timestamp within the correction window
        meal_ts = data[(data['glucose_level_ts'] <= bolus_start_ts) & (bolus_start_ts - data['glucose_level_ts'] <= pd.Timedelta(minutes=correction_window[1]))]['meal_ts'].max()
        
        # Check if it's a correction dose based on the filtered glucose_sliced_datetime
        if is_correction_dose(glucose_sliced_datetime[0], meal_ts, correction_window):
            glucose_sliced_segments.append(glucose_sliced)
            glucose_seg_ts_segments.append(glucose_seg_ts)

In [381]:
        
# Check if any segments were found
if len(glucose_sliced_segments) > 0:
    # Plotting code
    fig, ax = plt.subplots(len(glucose_sliced_segments), 1, figsize=(12, 6 * len(glucose_sliced_segments)))
    fig.tight_layout()

    for i, segment in enumerate(glucose_sliced_segments):
        ax[i].grid()
        ax[i].plot(glucose_seg_ts_segments[i], segment)
        ax[i].plot(glucose_seg_ts_segments[i][peaks[i]], segment[peaks[i]], "yx")
        ax[i].plot(glucose_seg_ts_segments[i][troughs[i]], segment[troughs[i]], "r.")
        ax[i].set_ylabel('glucose level (mg/dL)')

        fmt = mdates.DateFormatter('%Y-%m-%d %H:%M:%S')
        ax[i].xaxis.set_major_formatter(fmt)
        ax[i].tick_params(axis='x', rotation=30)

    plt.xlabel('Timestamp')
    plt.show()
else:
    print("No segments found.")

No segments found.
